In [135]:

import sqlite3
import os
import pandas as pd
from datetime import datetime, time, timedelta

In [136]:
# Read the data from the xlsx file
df = pd.read_excel('/Users/dongthan/Library/CloudStorage/OneDrive-VietJetAviationJointStockCompany/Work package/OWP STATISTIC/OWP_STATISTIC_ON_24112024.xlsx', sheet_name='MOC INPUT')

In [138]:
from datetime import timedelta, time
import pandas as pd

# Bước 1: Thiết lập hàng đầu tiên làm tên cột
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)  # Reset index sau khi bỏ hàng đầu

# Bước 2: Điền giá trị NaN trong cột 'STATUS' với 'ACTIVE'
df['STATUS'].fillna('ACTIVE', inplace=True)

# Bước 3: Danh sách các cột để điền NaN bằng 0
columns_to_fill = [
    'MPWR_MEC', 'MHRS_MEC', 'MPWR_A', 'MHRS_A', 'MPWR_B1', 'MHRs_B1', 'MPWR_B2', 'MHRs_B2',
    'MPWR_BSI', 'MHRs_BSI', 'MPWR_ENGR', 'MHRs_ENGR', 'MPWR_NDT', 'MHRs_NDT', 
    'MPWR_STRUC', 'MHRs_STRUC', 'MPWR_CABIN', 'MHRs_CABIN','STA','STD','GRD_TIME'
]

# Điền NaN trong các cột chỉ định bằng 0
df[columns_to_fill] = df[columns_to_fill].fillna(0)

# Bước 4: Các cột MHrs cần chuyển đổi
time_columns = ['STA','STD','GRD_TIME','MHRS_MEC', 'MHRS_A', 'MHRs_B1', 'MHRs_B2', 'MHRs_BSI', 'MHRs_ENGR', 'MHRs_NDT', 'MHRs_STRUC', 'MHRs_CABIN']

# Bước 5: Chuyển đổi các cột MHRs_ trong time_columns sang minutes
def convert_time_to_minutes(time_obj):
    if isinstance(time_obj, pd.Timestamp):
        time_obj = time_obj.time()
    if isinstance(time_obj, time):
        return time_obj.hour * 60 + time_obj.minute
    return time_obj  # Return the value as is if it's not a time object

# Áp dụng hàm chuyển đổi cho các cột chỉ định
df[time_columns] = df[time_columns].applymap(convert_time_to_minutes)

df

,STATUS,AC,STATION,STA,STD,GRD_TIME,TOTAL TASK,MPWR_MEC,MHRS_MEC,MPWR_A,...,MPWR_BSI,MHRs_BSI,MPWR_ENGR,MHRs_ENGR,MPWR_NDT,MHRs_NDT,MPWR_STRUC,MHRs_STRUC,MPWR_CABIN,MHRs_CABIN
0,C-CHECK,A689,CGK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A-CHECK,A810,CGK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A-CHECK,A811,CGK,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACTIVE,A642,CXR,20,205,185,3,1,60,1,...,0,0,0,0,0,0,0,0,0,0
4,ACTIVE,A648,CXR,1005,1080,75,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,ACTIVE,A814,SGN,950,1170,220,8,0,0,1,...,0,0,0,0,0,0,0,0,0,0
84,ACTIVE,A816,SGN,380,665,285,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
85,ACTIVE,A817,SGN,305,400,95,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
86,C-CHECK,A697,TSN,,,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
# Lọc TOTAL TASK <> 0
df = df[df['TOTAL TASK'] != 0]
df

,STATUS,AC,STATION,STA,STD,GRD_TIME,TOTAL TASK,MPWR_MEC,MHRS_MEC,MPWR_A,...,MPWR_BSI,MHRs_BSI,MPWR_ENGR,MHRs_ENGR,MPWR_NDT,MHRs_NDT,MPWR_STRUC,MHRs_STRUC,MPWR_CABIN,MHRs_CABIN
3,ACTIVE,A642,CXR,20,205,185,3,1,60,1,...,0,0,0,0,0,0,0,0,0,0
4,ACTIVE,A648,CXR,1005,1080,75,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,ACTIVE,A649,CXR,1220,1365,145,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,ACTIVE,A667,CXR,250,370,120,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,ACTIVE,A670,CXR,1275,5,170,8,1,60,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,ACTIVE,A691,SGN,1240,1335,95,3,0,0,1,...,0,0,0,0,0,0,0,0,0,0
82,ACTIVE,A694,SGN,155,420,265,12,2,120,1,...,0,0,0,0,0,0,0,0,0,0
83,ACTIVE,A814,SGN,950,1170,220,8,0,0,1,...,0,0,0,0,0,0,0,0,0,0
86,C-CHECK,A697,TSN,,,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
# DF_SGN = df[df['STATION'] == 'SGN']

df_sgn = df[df['STATION'] == 'SGN']

df_sgn


,STATUS,AC,STATION,STA,STD,GRD_TIME,TOTAL TASK,MPWR_MEC,MHRS_MEC,MPWR_A,...,MPWR_BSI,MHRs_BSI,MPWR_ENGR,MHRs_ENGR,MPWR_NDT,MHRs_NDT,MPWR_STRUC,MHRs_STRUC,MPWR_CABIN,MHRs_CABIN
47,STORAGE,A521,SGN,1020,300,720,5,1,60,0,...,0,0,0,0,0,0,0,0,0,0
48,ACTIVE,A522,SGN,1380,580,640,3,1,60,1,...,0,0,0,0,0,0,0,0,0,0
49,ACTIVE,A523,SGN,1385,100,155,3,1,60,1,...,0,0,0,0,0,0,0,0,0,0
51,ACTIVE,A527,SGN,1315,425,550,5,1,60,1,...,0,0,0,0,0,0,1,90,0,0
52,ACTIVE,A528,SGN,1390,80,130,2,1,60,1,...,0,0,0,0,0,0,0,0,0,0
53,STORAGE,A531,SGN,1020,300,720,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54,ACTIVE,A532,SGN,1360,345,425,7,1,60,1,...,0,0,0,0,0,0,1,60,0,0
55,ACTIVE,A538,SGN,1400,160,200,2,1,60,1,...,0,0,0,0,0,0,0,0,0,0
56,STANDBY,A542,SGN,50,0,0,2,1,60,1,...,0,0,0,0,0,0,0,0,0,0
57,STANDBY,A544,SGN,1295,,0,7,1,60,1,...,0,0,0,0,0,0,2,120,0,0


In [148]:
import pandas as pd

# Function to calculate total MHRs for a given category
def calculate_total_mhrs(df, mhrs_col, mpwr_col):
    return (df[mhrs_col] * df[mpwr_col]).sum()

# List of categories and their corresponding columns
categories = ['MEC', 'A', 'B1', 'B2', 'BSI', 'ENGR', 'NDT', 'STRUC', 'CABIN']
mhrs_columns = ['MHRS_MEC', 'MHRS_A', 'MHRs_B1', 'MHRs_B2', 'MHRs_BSI', 'MHRs_ENGR', 'MHRs_NDT', 'MHRs_STRUC', 'MHRs_CABIN']
mpwr_columns = ['MPWR_MEC', 'MPWR_A', 'MPWR_B1', 'MPWR_B2', 'MPWR_BSI', 'MPWR_ENGR', 'MPWR_NDT', 'MPWR_STRUC', 'MPWR_CABIN']

# Calculate total MHRs for each category
total_mhrs = [calculate_total_mhrs(df_sgn, mhrs_col, mpwr_col) for mhrs_col, mpwr_col in zip(mhrs_columns, mpwr_columns)]

# Create DataFrame with the results
total_sgn_mhrs_df = pd.DataFrame({
    'Category': categories,
    'Total MHRs': total_mhrs
})

# Add column for Total MHrs in hours
total_sgn_mhrs_df['Total MHrs'] = total_sgn_mhrs_df['Total MHRs'] / 60

total_sgn_mhrs_df

,Category,Total MHRs,Total MHrs
0,MEC,2640,44.0
1,A,3030,50.5
2,B1,2550,42.5
3,B2,660,11.0
4,BSI,0,0.0
5,ENGR,0,0.0
6,NDT,0,0.0
7,STRUC,450,7.5
8,CABIN,180,3.0
